# 09. Cleaning Data - Casting Datatypes and Handling Missing Values

---

In [2]:
import pandas as pd
import numpy as np

In the last notebook, we looked at how aggreagate functions work in Pandas, and we learned about grouping. In this notebook, we go into cleaning data and handling missing values within datasets, which is an essential step in examining any dataset.

We will look at how this can apply to a smaller dataset first, before moving on to a larger one. For that, we bring back our small "people" datset from the previous notebooks.

As we need to learn how to work with missing values within data, such as **NaN, NA, "missing", None**, an so on.. Let's first put some of these in our dataset:

In [19]:
people = {'First name': ['Adam', 'John', 'Jake', 'Jane', 'NA', 'Brock', 'Brian', None],
         'Last name': ['Missing', 'Doe', np.nan, 'Snow', 'Moe', 'Moe', 'Moe', np.nan],
         'Email': ['adamsmith@gmail.com', 'NA', 'jakedoe@notawebsite.org', None, 
                   'JobM@mail.com', None, np.nan, None],
         'Age': ['52', '34', '21', '28', '59', '40', '44', np.nan]}

In [21]:
people_df = pd.DataFrame(people)

In [5]:
people_df

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NA,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0
7,None,NaN,None,NaN


**Depending on what kind of data we have, and what we want to achieve with it, we are going to have different options to pick from when it comes to missing data.**

One of our options will be to simply remove this data. For this, we can use the **dropna** method:

In [6]:
people_df.dropna()

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
4,NA,Moe,JobM@mail.com,59.0


We see that this drops most of the rows of the DataFrame. This is because we have either one or more **np.nan** (NaN) or "None" values within each of these rows. We can extend these dropped values to includes custom ones such as "NA" or "missing" and so on, but first, let's look at the default arguments of **dropna** and how we can change them according to our needs:

In [7]:
people_df.dropna(axis='index', how='any')

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
4,NA,Moe,JobM@mail.com,59.0


We see that we have 2 arguments here:

**axis** can take a value of either **0 or "index"** or **1 or "columns"**. Decides whether we want to remove the rows which include our missing values or the columns.

**how** can take a value of either **any** or **all**. Decides whether removing a row/column should require the existence of any (one at least) missing values, or if it should require all the values within that row/column to be missing.

To show this in effect, let's change the **how** arguent from the example above:

In [8]:
people_df.dropna(axis='index', how='all')

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NA,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0


As we see, this time only one row was removed, which is the last row containing all missing values.

We could also change the **axis** index, but this would result in an empty DataFrame here, as one of the rows has all missing values, which mean that each column has at least one missing value.

We sometimes need to think in a less wasteful manner, in order to save as much useful data as possible. This, of course, depends on what is useful exactly in that specific case.

To give an example, let's assume that we need to send emails to the people within our DataFrame, and that without the ability to address a person by sending them an email, the rest of their data is not useful to us.

In that case, we can ignore missing values in a row if they occur in any column, except for "Email", which mean that we want to drop any row which has a missing value in the email column.

For that, we use the same **dropna** method we just used, but we use a third argument: **subset**.

**subset** can be used to **specify which columns to limit checking for missing values within**. It can take a single string or a list of strings. For instance:

In [9]:
people_df.dropna(axis='index', how='any', subset='Email')

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
4,NA,Moe,JobM@mail.com,59.0


We see that this kept the second row within our DataFrame, which has a custom missing value "NA" in the Email column. We'll see how to deal with custom values as missing values  in a bit.

**Note: since we are considering only one column in the example above, changing the "how" argument from "any" to "all" would not make a difference.**

Combining the use of **how** and **subset** can make for some useful use cases. For example, we can consider a scenario where we can keep a person within our dataset, but only if we have either their last name or email. In other words, both cannot be missing, otherwise we don't need this row anymore.

For that, we pass the 2 mentioned columns to our **subset** argument, and change the **how** argument to **all**, which means that we now want to drop rows where all (in this case both) of the mentioned columns in **subset** are missing:

In [10]:
people_df.dropna(axis='index', how='all', subset=['Email', 'Last name'])

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NA,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0


**Note: As with many other methods within Pandas, dropna needs an an inplace=True flag for the effects to take place.**

In [11]:
people_df

,First name,Last name,Email,Age
0,Adam,Missing,adamsmith@gmail.com,52.0
1,John,Doe,NA,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NA,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0
7,None,NaN,None,NaN


Now, as we mentioned before, we can expand the list of values that we treat as missing values. For example, if the people who created the dataset we're working with used a different convention to indicate that a value was missing, like we've done here with "Missing" and "NA". But how do we filter these  and/or similar custom missing values from our datasets?

Since we have manually created the people DataFrame that we're applying these steps on for now, we can simply replace the custom missing values we've added with "None" or "NaN", and then they can be filtered out along with the default missing values during our cleaning process:

In [13]:
people_df.replace('NA', np.nan, inplace=True)
people_df.replace('Missing', np.nan, inplace=True)

In [14]:
people_df

,First name,Last name,Email,Age
0,Adam,NaN,adamsmith@gmail.com,52.0
1,John,Doe,NaN,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NaN,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0
7,None,NaN,None,NaN


Using te steps we've used above would now affect all fields. For instance:

In [16]:
people_df.dropna(how='all')

,First name,Last name,Email,Age
0,Adam,NaN,adamsmith@gmail.com,52.0
1,John,Doe,NaN,34.0
2,Jake,NaN,jakedoe@notawebsite.org,21.0
3,Jane,Snow,None,28.0
4,NaN,Moe,JobM@mail.com,59.0
5,Brock,Moe,None,40.0
6,Brian,Moe,NaN,44.0


In practice, we load our datasets from an external source. We'll see how we can filter out custom missing values below, when we start cleaning the Stack Overflow dataset.

Now, instead of applying a filtering step to our dataset, we might want to see which of the fields within our dataset is classified as a missing value. For that, we can use **isna**, which works as follows:

In [17]:
people_df.isna()

,First name,Last name,Email,Age
0,False,True,False,False
1,False,False,True,False
2,False,True,False,False
3,False,False,True,False
4,True,False,False,False
5,False,False,True,False
6,False,False,True,False
7,True,True,True,True


As we see, we get a map of booleans, which is True in the fields with missing value.

Now, sometimes in practice we work with numerical datasets, and we want to fill the missing values within these datasets with some value.

Let's say for example, that we have a dataset of students' grades in some assignments. Some of the fields might be missing values because some students never submitted a solution for some of the assignments. We might want to grade these with a zero, or we might want to mark them with a specific value, so that we calculate a grade for them using some other mechanism later.

In any case, the **fillna** method comes in handy in such situations. Let's say we want to fill all of our missing fields here with a bold word: "MISSING":

In [18]:
people_df.fillna('MISSING')

,First name,Last name,Email,Age
0,Adam,MISSING,adamsmith@gmail.com,52.0
1,John,Doe,MISSING,34.0
2,Jake,MISSING,jakedoe@notawebsite.org,21.0
3,Jane,Snow,MISSING,28.0
4,MISSING,Moe,JobM@mail.com,59.0
5,Brock,Moe,MISSING,40.0
6,Brian,Moe,MISSING,44.0
7,MISSING,MISSING,MISSING,MISSING


**Note: We can pass a string or a number to fillna, which makes it useful for textual and for numerical data as well.**

**Note: inplace=True is required to make the changes permanent.**

We've seen so far how we can deal with missing data. But one other thing that we need to keep an eye on when analyzing a dataset in data types. One example is our "age" column here. Although age is a number, we created our people DataFrame's age column using strings which contains the numbers. This can cause problems if we wish to do some numerical analysis on this column.

To deal with such data types problems, we may want to look at all data types available in our DataFrame first. For that, we use the **.dtypes** attribute:

In [22]:
people_df.dtypes

First name    object
Last name     object
Email         object
Age           object
dtype: object

We see the word "object" here a lot. This means the column is either filled with strings or is a mixture of data types.

We might think that in our example, it would be fitting if we cast our "age" column to integers, as that is the most fitting type for age. But since we have some NaN values within this column, which are actually of the float type, and cannot be converted, it would seem that we have to cast the column to float instead. This can be done using the **astype** method:

In [28]:
people_df['Age'] = people_df['Age'].astype(float)

**Note: Casting the column to integers would have been possible if it had no NaN values within it. An alternative solution would be to fill the NaN field with a 0 using the fillna method, which would allow us to cast the column to integers after that. However, zero is a valid value, which would affect the aggregate functions we might want to run on the column. So casting to floats is probably the safest option.**

This changes the type of the "age" column, which we can see using the **dtypes** attribute:

In [30]:
people_df.dtypes

First name     object
Last name      object
Email          object
Age           float64
dtype: object

Now, we can run numerical anlysis methods on this column. For example, we can find the mean age of the people in our DataFrame:

In [29]:
people_df['Age'].mean()

39.714285714285715

**Note: Suppose we want to cast our entire dataset to a specific type, like with a numerical dataset, then we can use the astype method directly on the DataFrame itself.**

-------------------------------------------------------------------------------------------------------------

Now, let's apply what we've learned so far on the Stack Overflow dataset:

First, we load our files, then we change the default options of maximum visible rows and colums to 85 each.

**This time, however, we're going to add something:**

We mentioned earlier that we can filter out custom missing values as we load our datasets from an external source. This occurs by passing a list of values which we'd like Pandas to treat as missing values to the loading method as we load our files. Here's a demonstration of how that works:

In [34]:
custom_missing_values = ['Missing', 'NA', 'MISSING']

df = pd.read_csv('survey_results_public.csv', na_values= custom_missing_values)
schema_df = pd.read_csv('survey_results_schema.csv')

This has the effect of making Pandas treat our custom missing values as missing values (like NaN) by default, leading to a quicker cleaning process.

In [32]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

Let's look at the head of our dataset again:

In [33]:
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or ...",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a jo...","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,Taken an online course in programming or softw...,NaN,"Developer, desktop or enterprise applications;...",NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,I am actively looking for a job,I've never had a job,NaN,NaN,Financial performance or funding status of the...,"Something else changed (education, award, medi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,NaN,MySQL,Windows,Windows,Django,Django,NaN,NaN,Atom;PyCharm,Windows,I do not use containers,NaN,Useful across many domains and could change ma...,Yes,Yes,Yes,Instagram,Online,Username,2017,Daily or almost daily,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,11-30 minutes,Yes,A few times per month or weekly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,"Taught yourself a new language, framework, or ...",100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,22,1,Slightly satisfied,Slightly satisfied,Not at all confident,Not sure,Not sure,"I’m not actively looking, but I am open to new...",1-2 years ago,Interview with people in peer roles,No,"Languages, frameworks, and other technolo

Now, with the Stack Overflow dataset we're looking at here, the creators have done a good job keeping it relatively free of missing values. So instead, let's look at an example of casting values.

Within this dataset, it would be difficult to calculate the average number of years of expierience among the participants. Let's examine why that us, and see how we can get that number.

First, let's look at the "YearsCode" column, which is the relevant column in this case:

In [35]:
df['YearsCode'].head(10)

0      4
1    NaN
2      3
3      3
4     16
5     13
6      6
7      8
8     12
9     12
Name: YearsCode, dtype: object

This initial examining looks promising. We have some numbers, as well as some NaN values.

So let's try grabbing the mean of this column:

In [37]:
# df['YearsCode'].mean()

# returns: "TypeError: can only concatenate str (not "int") to str"

We see that we get a type error. Maybe we can solve this by casting it to float first?

In [39]:
# df['YearsCode'] = df['YearsCode'].astype(float)

# returns: "ValueError: could not convert string to float: 'Less than 1 year'"

Hmm.. This also triggers an error, as the answer "Less than 1 year" obviously can't be turned into a float number directly.

It looks like there are more values in there than simply numbers and NaN values. Let's look at all the unique values that occur in this column.

We previously saw how we can achieve that with the **value_counts** method, but here we don't want to count the values, we just want to identify the uique values within the column:

In [40]:
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 'Less than 1 year', '30', '9', '26', '40', '19',
       '15', '20', '28', '25', '1', '22', '11', '33', '50', '41', '18',
       '34', '24', '23', '42', '27', '21', '36', '32', '39', '38', '31',
       '37', 'More than 50 years', '29', '44', '45', '48', '46', '43',
       '47', '49'], dtype=object)

As we see, there are 2 strings within the column, which we will need to replace if we want to be able to apply aggregate functions to this column.

At the risk of slightly changing the actual average, let's replace the two strings with numbers.
We'll pick 0 for the first one, and 51 for the second:

In [41]:
df['YearsCode'].replace('Less than 1 year', 0, inplace=True)
df['YearsCode'].replace('More than 50 years', 51, inplace=True)

Now, let's try to get the mean number of years of experience, by first casting the column to floats to deal with NaN values, and then using the **mean** method:

In [44]:
df['YearsCode'] = df['YearsCode'].astype(float)
df['YearsCode'].mean()

11.662114216834588

As we see, it is now possible to apply aggregate functions to the column. Let's try to find the median number:

In [45]:
df['YearsCode'].median()

9.0

With that, we can now perform data cleaning steps and data casting steps on datasets in the real world.